TODO make it more useful for use in other scripts:
- make it possible to select specific joints
- make it possible to select motion types / transition between motion types
- add plots

# Current Consumption Example

The current consumption example demonstartes how to get the values of the cunsumed current of all frames where joint movement was logged.

## Setup

### Goggl Colab

The following installations of requirements should only be run in google colab.
When you run this notbook on your machine, the requirements should be installed in your local venv.

In [ ]:
!pip install numpy naoth

### Local venv

*assuming you have python>=3.6.9 and an active venv*
1. install the requirements `python -m pip install -r requirements.txt`
1.1 in `/py/naoth` install naoth via `python -m pip install .`
2. start the jupyter server `python -m jupyter notebook notebooks`
3. have fun with the notebooks in notebooks/

## Usage

In [22]:
import numpy as np
from naoth.log import BehaviorParser
from naoth.log import Reader as LogReader
from naoth.datasets import motion

In [23]:
JointID = {
    "HeadPitch": 0,
    "HeadYaw": 1,
    "RShoulderRoll": 2,
    "LShoulderRoll": 3,
    "RShoulderPitch": 4,
    "LShoulderPitch": 5,
    "RElbowRoll": 6,
    "LElbowRoll": 7,
    "RElbowYaw": 8,
    "LElbowYaw": 9,
    "RHipYawPitch": 10,
    "LHipYawPitch": 11,
    "RHipPitch": 12,
    "LHipPitch": 13,
    "RHipRoll": 14,
    "LHipRoll": 15,
    "RKneePitch": 16,
    "LKneePitch": 17,
    "RAnklePitch": 18,
    "LAnklePitch": 19,
    "RAnkleRoll": 20,
    "LAnkleRoll": 21
}

In [24]:
def frame_filter(behavior_parser, frame):
    try:
        behavior_frame = behavior_parser.parse(frame['BehaviorStateSparse'])

        return [frame["FrameInfo"].time / 1000.0,
                frame["BodyStatus"].currentSum[JointID["RHipYawPitch"]],
                frame["BodyStatus"].currentSum[JointID["LHipYawPitch"]],
                frame["BodyStatus"].currentSum[JointID["RHipPitch"]],
                frame["BodyStatus"].currentSum[JointID["LHipPitch"]],
                frame["BodyStatus"].currentSum[JointID["RHipRoll"]],
                frame["BodyStatus"].currentSum[JointID["LHipRoll"]],
                frame["BodyStatus"].currentSum[JointID["RKneePitch"]],
                frame["BodyStatus"].currentSum[JointID["LKneePitch"]],
                frame["BodyStatus"].currentSum[JointID["RAnklePitch"]],
                frame["BodyStatus"].currentSum[JointID["LAnklePitch"]],
                frame["BodyStatus"].currentSum[JointID["RAnkleRoll"]],
                frame["BodyStatus"].currentSum[JointID["LAnkleRoll"]],
                behavior_frame.input_symbols['executed_motion.type'].value
                ]

    except KeyError:
        raise StopIteration

Get the logfile data as numpy array.

In [25]:
logfile_name = motion.load_data('cognition')
frame_list = list()

with LogReader(logfile_name) as logfile:
    behavior_parser = BehaviorParser()
    for frame in logfile.read():
        if 'BehaviorStateComplete' in frame:
            behavior_parser.initialize(frame['BehaviorStateComplete'])
        frame_list.append(frame_filter(behavior_parser, frame))

# make an numpy array
data = np.array(frame_list)

In [26]:
new_data = np.zeros((data.shape[0] - 1, data.shape[1]))

for idx in range(data.shape[0] - 1):
    # calculate the frame time since last frame
    time_diff = data[idx + 1, 0] - data[idx, 0]
    # calculate the difference between the current sums
    current_difference = data[idx + 1, 1:13] - data[idx, 1:13]

    new_data[idx, 0] = time_diff
    new_data[idx, 1:13] = current_difference
    new_data[idx, 13] = data[idx, 13]  # leave executed motion type as it is

Extract the frames where the robot was standing or walking.

In [31]:
stand = np.array([frame for frame in new_data if int(frame[13]) == 4])
walk = np.array([frame for frame in new_data if int(frame[13]) == 5])

Make some simple statistics on it.

Print the time the robot was standing with how much current was used.

In [32]:
time_stand = sum(stand[:, 0]) if list(stand) else 0
consumption_r_hip_yaw_pitch_stand = sum(stand[:, 1]) if list(stand) else 0
print(f"time standing: {time_stand} seconds with {consumption_r_hip_yaw_pitch_stand} current used at RHipYawPitch")

time standing: 0 seconds with 0 current used at RHipYawPitch


Print the time the robot was walking with how much current was used.

In [33]:
time_walk = sum(walk[:, 0]) if list(walk) else 0
consumption_r_hip_yaw_pitch_walk = sum(walk[:, 1]) if list(walk) else 0
print(f"time walking : {time_walk} seconds with {consumption_r_hip_yaw_pitch_walk} current used at RHipYawPitch")

time walking : 6.692999999999984 seconds with 5.008000157773495 current used at RHipYawPitch
